In [ ]:
CREATE TABLE IF NOT EXISTS DIM_DIM_MEMBERS (
    ID_MEMBER      NUMBER(38,0),
    ID_CITY        NUMBER(38,0),
    ID_GROUP       NUMBER(38,0),
    MEMBER_NAME    VARCHAR,
    MEMBER_STATUS  VARCHAR,
    BIO            VARCHAR,
    VISITED        TIMESTAMP_NTZ,
    HOMETOWN       VARCHAR,
    JOINED         TIMESTAMP_NTZ,
    LINK           VARCHAR,
    PRIMARY KEY (ID_MEMBER, ID_GROUP)  -- declarativo (Snowflake no lo hace cumplir)
);

MERGE INTO DATA_SLV.DIM_DIM_MEMBERS AS d
USING (
  SELECT DISTINCT
    TRY_TO_NUMBER(m.MEMBER_ID)                                  AS ID_MEMBER,
    TRY_TO_NUMBER(c.ID_CITY)                                    AS ID_CITY,   -- si CITY_ID es texto en DIM_CITIES
    TRY_TO_NUMBER(m.GROUP_ID)                                   AS ID_GROUP,
    NULLIF(TRIM(m.MEMBER_NAME),   '')                           AS MEMBER_NAME,
    NULLIF(TRIM(m.MEMBER_STATUS), '')                           AS MEMBER_STATUS,
    NULLIF(TRIM(m.BIO),           '')                           AS BIO,
    m.VISITED                                                   AS VISITED,
    NULLIF(TRIM(m.HOMETOWN),      '')                           AS HOMETOWN,
    m.JOINED                                                    AS JOINED,
    NULLIF(TRIM(m.LINK),          '')                           AS LINK
  FROM MEETUP_DB.DATA_BRZ.MEMBERS AS m
  LEFT JOIN DATA_SLV.DIM_CITIES AS c
    ON UPPER(TRIM(c.CITY))    = UPPER(TRIM(m.CITY))
   AND COALESCE(UPPER(TRIM(c.STATE))  , '') = COALESCE(UPPER(TRIM(m.STATE))  , '')
   AND COALESCE(UPPER(TRIM(c.COUNTRY)), '') = COALESCE(UPPER(TRIM(m.COUNTRY)), '')
  WHERE TRY_TO_NUMBER(m.MEMBER_ID) IS NOT NULL
    AND TRY_TO_NUMBER(m.GROUP_ID)  IS NOT NULL
) AS s
ON  d.ID_MEMBER = s.ID_MEMBER
AND d.ID_GROUP  = s.ID_GROUP

WHEN MATCHED THEN UPDATE SET
  d.ID_CITY        = s.ID_CITY,
  d.MEMBER_NAME    = s.MEMBER_NAME,
  d.MEMBER_STATUS  = s.MEMBER_STATUS,
  d.BIO            = s.BIO,
  d.VISITED        = s.VISITED,
  d.HOMETOWN       = s.HOMETOWN,
  d.JOINED         = s.JOINED,
  d.LINK           = s.LINK

WHEN NOT MATCHED THEN INSERT (
  ID_MEMBER, ID_CITY, ID_GROUP, MEMBER_NAME, MEMBER_STATUS, BIO, VISITED, HOMETOWN, JOINED, LINK
) VALUES (
  s.ID_MEMBER, s.ID_CITY, s.ID_GROUP, s.MEMBER_NAME, s.MEMBER_STATUS, s.BIO, s.VISITED, s.HOMETOWN, s.JOINED, s.LINK
);